# Knapsack Problem

Description of the problem (from [Wikipedia](https://en.wikipedia.org/wiki/Knapsack_problem)):

> Given a set of items, each with a weight and a value, determine which items to include in the collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

Example:

<table>
    <thead>
        <th>Item</th>
        <th>Weight</th>
        <th>Value</th>
    </thead>
    <tbody>
        <tr>
            <td>A</td>
            <td>3</td>
            <td>4</td>
        </tr>
        <tr>
            <td>B</td>
            <td>2</td>
            <td>2</td>
        </tr>
        <tr>
            <td>C</td>
            <td>7</td>
            <td>10</td>
        </tr>
        <tr>
            <td>D</td>
            <td>4</td>
            <td>5</td>
        </tr>
        <tr>
            <td>E</td>
            <td>5</td>
            <td>6</td>
        </tr>
    </tbody>
</table>

Knapsack Capacity:

<table>
    <thead>
        <th>Capacity</th>
    </thead>
    <tbody>
        <tr><td>11</td></tr>
    </tbody>
</table>


In [1]:
# Problem Constants
ITEMS = [
    { "name": "A", "weight": 3, "value": 4 },
    { "name": "B", "weight": 2, "value": 2 },
    { "name": "C", "weight": 7, "value": 10 },
    { "name": "D", "weight": 4, "value": 5 },
    { "name": "E", "weight": 5, "value": 6 },
]

KNAPSACK_CAPACITY = 11

# Ultility Methods
def select_items_until_capacity(items):
    selected_items = []
    weight = 0
    for item in items:
        if weight + item["weight"] <= KNAPSACK_CAPACITY:
            selected_items.append(item)
            weight += item["weight"]
    return selected_items

def solution_summary(solution_items):
    items = [item["name"] for item in solution_items]
    load = sum([item["weight"] for item in solution_items])
    value = sum([item["value"] for item in solution_items])
    return {
        "items": items,
        "load": f"{load:.2f}",
        "value": f"{value:.2f}",
        "ksnapsack_usage": f"{(load/KNAPSACK_CAPACITY)*100:.2f}%"
    }

In [2]:
# First attempt: Try to select the most lighther items until we achieve the capacity
items_sorted_by_weight = sorted(ITEMS, key=lambda i: i["weight"])
s1_items = select_items_until_capacity(items_sorted_by_weight)
solution_summary(s1_items)

{'items': ['B', 'A', 'D'],
 'load': '9.00',
 'value': '11.00',
 'ksnapsack_usage': '81.82%'}

In [3]:
# Second attempt: Try to select the most valuable items until we achieve the capacity
items_sorted_by_value = sorted(ITEMS, key=lambda i: i["value"], reverse=True)
s2_items = select_items_until_capacity(items_sorted_by_value)
solution_summary(s2_items)

{'items': ['C', 'D'],
 'load': '11.00',
 'value': '15.00',
 'ksnapsack_usage': '100.00%'}

In [4]:
# Third attempt: Try to select the most profitable (relation between value and weight) items until we achieve the capacity
items_sorted_by_profit = sorted(ITEMS, key=lambda i: i["value"]/i["weight"], reverse=True)
s3_items = select_items_until_capacity(items_sorted_by_profit)
solution_summary(s3_items)

{'items': ['C', 'A'],
 'load': '10.00',
 'value': '14.00',
 'ksnapsack_usage': '90.91%'}

## Linear Programming Mathematical Model

**Definitions:**

Constants:
$$
\begin{aligned}
    N & : \text{Total number of items} \\
    W & : \text{Maximum capacity of the knapsack (maximum weight)}
\end{aligned}
$$

Parameters:
$$
\begin{aligned}
    i & : \text{Index of an item, where } i = 1, 2, \dots, N \\
    v_i & : \text{Value of item } i \\
    w_i & : \text{Weight of item } i
\end{aligned}
$$

Variables:
$$
\begin{aligned}
    x_i & =
        \begin{cases}
            1 & \text{if item } i \text{ is included in the knapsack} \\
            0 & \text{otherwise}
        \end{cases}
\end{aligned}
$$


**Constraints:**

Total weight of selected items must not exceed the knapsack capacity:
$$
\sum_{i=1}^{N} w_i x_i \leq W
$$

Each item can either be included (1) or not (0):
$$
    x_i \in \{0, 1\}
$$


**Goal:**

Maximize the total value of selected items:
$$
    \text{Maximize} \sum_{i=1}^{N} v_i x_i
$$


In [5]:
# PYOMO
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Contant N: Total number of items
model.N = pyo.Param(initialize=len(ITEMS)) # Actualy not needed

# Constant W: Maximum capacity of the knapsack (maximum weight)
model.W = pyo.Param(initialize=KNAPSACK_CAPACITY)

# Set I = {A, B, C, D, E}
model.I = pyo.Set(initialize=[item["name"] for item in ITEMS])

# Parameter v: Value of an item
model.v = pyo.Param(model.I, initialize={item["name"]: item["value"] for item in ITEMS})

# Parameter w: Weight of an item
model.w = pyo.Param(model.I, initialize={item["name"]: item["weight"] for item in ITEMS})

# Variable x: Defined if an item i is in the knapsack or not (1 or 0)
model.x = pyo.Var(model.I, within=pyo.Binary)

# Constraint 1: Total weight of selected items must not exceed the knapsack capacity
def capacity_constraint(model):
    return sum(model.w[i] * model.x[i] for i in model.I) <= model.W

model.capacity_constraint = pyo.Constraint(rule=capacity_constraint)

# Goal: Maximize the total value of selected items
def objective_function(model):
    return sum(model.v[i] * model.x[i] for i in model.I)

model.objective_function = pyo.Objective(rule=objective_function, sense=pyo.maximize)

# Instantiate Highs persistent solver (install highspy)
solver = pyo.SolverFactory("appsi_highs")

solver.solve(model, tee=True)

pyomo_selected_items = [item for item in ITEMS if model.x.extract_values()[item["name"]] == 1]
solution_summary(pyomo_selected_items)

Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
RUN!
MIP  has 1 rows; 5 cols; 5 nonzeros; 5 integer variables (5 binary)
Coefficient ranges:
  Matrix [2e+00, 7e+00]
  Cost   [2e+00, 1e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+01, 1e+01]
Presolving model
1 rows, 5 cols, 5 nonzeros  0s
1 rows, 5 cols, 5 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   1 rows
   5 cols (5 binary, 0 integer, 0 implied int., 0 continuous)
   5 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol  

{'items': ['C', 'D'],
 'load': '11.00',
 'value': '15.00',
 'ksnapsack_usage': '100.00%'}